<a href="https://colab.research.google.com/github/haker816/aman/blob/main/IPL_Score_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import keras
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# Loading Data
ipl=pd.read_csv("/content/ipl_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/ipl_data.csv'

In [ ]:
# Count of rows and columns
ipl.shape

In [ ]:
# view of dataframe
ipl.tail()

In [ ]:
# dropping unnecessary columns
df=ipl.drop(["mid","date","runs_last_5","wickets_last_5","runs","wickets","overs","striker","non-striker"],axis=1)

In [ ]:
# Separating dependent and independent data
X=df.iloc[:,:-1]
y=df["total"]

In [ ]:
# Doing Label encoding for categorical varibale like venue, batting_team, bowler_team etc
venue_encoder = LabelEncoder()
batting_team_encoder = LabelEncoder()
bowling_team_encoder = LabelEncoder()
striker_encoder = LabelEncoder()
bowler_encoder = LabelEncoder()


X['venue'] = venue_encoder.fit_transform(X['venue'])
X['bat_team'] = batting_team_encoder.fit_transform(X['bat_team'])
X['bowl_team'] = bowling_team_encoder.fit_transform(X['bowl_team'])
X['batsman'] = striker_encoder.fit_transform(X['batsman'])
X['bowler'] = bowler_encoder.fit_transform(X['bowler'])

In [ ]:
X.head()

In [ ]:
# Train test split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=24)

In [ ]:
# Scaling to make all numerical variable in one scale
scaler=MinMaxScaler()
X_train_s=scaler.fit_transform(X_train)
X_test_s=scaler.transform(X_test)

In [ ]:
X_train_s[0]

In [ ]:
# Define the neural network model
model = keras.Sequential([
	keras.layers.Input( shape=(X_train_s.shape[1],)),
	keras.layers.Dense(512, activation='relu'),
	keras.layers.Dense(216, activation='relu'),
	keras.layers.Dense(1, activation='linear')
])

# Compile the model with Huber loss
huber_loss = tf.keras.losses.Huber(delta=1.0)
model.compile(optimizer='adam', loss=huber_loss)

In [ ]:
# Train the model
model.fit(X_train_s, y_train, epochs=50, batch_size=64, validation_data=(X_test_s, y_test))


In [ ]:
# Plotting loss graph
model_losses = pd.DataFrame(model.history.history)
model_losses.plot()

In [ ]:
# Getting sccore for our model
y_pred=model.predict(X_test_s)
mean_absolute_error(y_test,y_pred)

In [ ]:
# Libraries for creating interactive widget
import ipywidgets as widgets
from IPython.display import display, clear_output

import warnings
warnings.filterwarnings("ignore")

In [ ]:
venue = widgets.Dropdown(options=df['venue'].unique().tolist(),description='Select Venue:')
batting_team = widgets.Dropdown(options =df['bat_team'].unique().tolist(), description='Select Batting Team:')
bowling_team = widgets.Dropdown(options=df['bowl_team'].unique().tolist(), description='Select Batting Team:')
#runs=widgets.IntText(value=0,description="Runs:",disabled=False)
striker = widgets.Dropdown(options=df["batsman"].unique().tolist(), description='Select Striker:')
bowler = widgets.Dropdown(options=df["bowler"].unique().tolist(), description='Select Bowler:')
#wickets=widgets.BoundedIntText(value=0,min=0,max=9,step=1,description='Wickets:',disabled=False)
#over=widgets.BoundedFloatText(value=5,min=0,max=20,step=0.1,description='Overs:',disabled=False)
#striker_run=widgets.IntText(value=0,description='Striker Runs:',disabled=False)
#non_striker_run=widgets.IntText(value=0,description='Non-Striker Runs:',disabled=False)

predict_button = widgets.Button(description="Predict Score")

def predict_score(b):
	with output:
		clear_output() # Clear the previous output


		# Decode the encoded values back to their original values
		decoded_venue = venue_encoder.transform([venue.value])
		decoded_batting_team = batting_team_encoder.transform([batting_team.value])
		decoded_bowling_team = bowling_team_encoder.transform([bowling_team.value])
		decoded_striker = striker_encoder.transform([striker.value])
		decoded_bowler = bowler_encoder.transform([bowler.value])


		input = np.array([decoded_venue, decoded_batting_team, decoded_bowling_team,decoded_striker, decoded_bowler])
		input = input.reshape(1,5)
		input = scaler.transform(input)
		#print(input)
		predicted_score = model.predict(input)
		predicted_score = int(predicted_score[0,0])

		print(predicted_score)


In [ ]:
predict_button.on_click(predict_score)
output = widgets.Output()
display(venue, batting_team, bowling_team, striker, bowler, predict_button, output)
